In [110]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from sklearn.preprocessing import normalize
from tensorflow.keras import regularizers
from tensorflow import keras
from scipy.signal import butter, filtfilt, iirnotch
import pickle
import sklearn
import pywt
import sys

sys.path.insert(0,"siggy_ml")
from erp_classifier import classifier

In [111]:
X = np.load(r"siggy_ml\X_test_data.npy",allow_pickle=True)
y = np.load(r"siggy_ml\y_test_data.npy",allow_pickle=True)

In [112]:
X.shape

(139, 6, 256)

In [113]:
for i in X[0]:
    print(i.shape)

(256,)
(256,)
(256,)
(256,)
(256,)
(256,)


In [114]:
X_n = []
for i in X:
    sample = []
    for j in i:
        if len(j)==108:
            sample.append(j)
            #print(len(j))
    # print(len(sample))
    # print("i loop")
    if len(sample)==6:
        X_n.append(sample)
np.asarray(X_n).shape

(0,)

In [131]:
MODEL = models.load_model(r"eeg_cnn")

In [116]:
MODEL.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 256, 6, 40)        1240      
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 256, 1, 40)        9640      
_________________________________________________________________
average_pooling2d_4 (Average (None, 16, 1, 40)         0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 640)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 80)                51280     
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 162       
Total params: 62,322
Trainable params: 62,322
Non-trainable params: 0
__________________________________________________

In [117]:
class real_classifier(classifier):
    def __init__(self, model, sampling_rate = 256, offset = 0.2):
      super().__init__(sampling_rate,offset)
      self.model = model

    def filter(self,data,low=0.5,high=60):
      return super().preprocess(data,low,high)

    def preprocess(self, data,y):

      xnew = np.linspace(-200,800,self.sampling_rate)
      
      X_n = []
      y_n = []
      # interpolate the data for consistent epoch lengths
      for i,k in zip(data,y):
        try:
          sample = []
          for j in i:
              xold = np.linspace(-200,800,len(j))
              print(len(j))
              sample.append(np.interp(xnew,xold,j))
          X_n.append(sample)
          y_n.append(k)
        except:
          pass
      X_n = np.asarray(X_n)

      X_clean = np.expand_dims(X_n,axis=-1)
      X_clean = np.swapaxes(X_clean,1,2)
      return X_clean,y_n


In [118]:
class cnn_classifier(real_classifier):

    def __init__(self, model, sampling_rate=256, offset=0.2):
        super().__init__(model, sampling_rate, offset)

    def predict(self, sample):
        return np.argmax(self.model.predict(sample),axis=1)

In [119]:
EPOCH_END = 600
ROI_START = 300
ROI_END = 500

In [120]:
def wavedec(data,t0=0,t1=EPOCH_END):
    coeffs = pywt.wavedec(data,"db8",level=8)
    cut = []
    for i in coeffs:
        x = np.linspace(-200,800,num=len(i))
        start = abs(x-t0).argmin()
        stop = abs(x-t1).argmin()
        cut.append(i[start:stop])
    x = np.linspace(-200,800,len(data))
    start = abs(x-t0).argmin()
    stop = abs(x-t1).argmin()
    cut.append(data[start:stop])
    return cut

In [121]:
class SVM_classifier(real_classifier):
    # to fix
    def __init__(self, model, sampling_rate = 256, offset = 0.2):
      super().__init__(model,sampling_rate,offset)

    def preprocess(self, data,y , lower = 0.5, upper = 60):
      X_clean,y_n = super().preprocess(data,y)
      filtered = self.filter(X_clean,lower,upper)[:,2,:]
      decomposed = [wavedec(i)[2] for i in filtered]

    def predict(self, sample):
        return self.clf.predict(sample)

In [122]:
svm_models = pickle.load(open("per_electrode.p", "rb"))

In [123]:
svm_clf = SVM_classifier(svm_models[2],256,0.2)

In [125]:
#xvm_x,xvmy = svm_clf.preprocess(X,y)

In [126]:
clf = cnn_classifier(models.load_model(r"eeg_cnn"),256,0.2)

In [127]:
X_test,y_test = clf.preprocess(X,y)

256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256
256


In [128]:
X_test[0].shape

(256, 6, 1)

In [129]:
clf.predict(X_test)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [130]:
y_test

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 Fa